# [3. Searching for Solutions](http://artint.info/2e/html/ArtInt2e.Ch3.html)

## About
This chapter casts the problem of an agent deciding how to solve a goal as the problem of searching to find a path in a graph.

## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter*. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

## [3.5.2 Depth-First Search](http://artint.info/2e/html/ArtInt2e.Ch3.S5.SS2.html)

- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=39) (page 39)

In depth-first search, the frontier acts like a LIFO (last-in, first-out) stack of paths. Using a stack means that the path selected and removed from the frontier at any time is the last path that was added. Depth-first search is appropriate when space is restricted, or when there are many solutions. On the other hand, depth-first search is not appropriate if it is possible to get stuck into infinite paths or if solutions exist at shallow depths.

In [ ]:
from aipython.searchProblem import Path
from aipython.searchGeneric import Frontier
from aispace2.jupyter.search import Displayable, visualize

class Searcher(Displayable):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    This does depth-first search unless overridden
    """
    def __init__(self, problem):
        """creates a searcher from a problem
        """
        self.problem = problem
        self.initialize_frontier()
        self.num_expanded = 0
        self.add_to_frontier(Path(problem.start_node()))
        super().__init__()

    def initialize_frontier(self):
        self.frontier = []
        
    def empty_frontier(self):
        return self.frontier == []
        
    def add_to_frontier(self,path):
        self.frontier.append(path)
        
    @visualize
    def search(self):
        """returns (next) path from the problem's start node
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):    # solution found
                self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                self.solution = path   # store the solution found
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(neighs):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")


In [ ]:
from aipython.searchProblem import simple_problem1, simple_problem2, edgeless_problem, cyclic_delivery_problem, acyclic_delivery_problem
from IPython.display import display

s = Searcher(simple_problem1)
s.sleep_time = 0.2
s.show_edge_costs = False
s.text_size = 15
s.search()
display(s)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
s.search()

## [3.6.1 A\* Search](http://artint.info/2e/html/ArtInt2e.Ch3.S6.SS1.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=41) (page 41)

A\* search uses both path cost and heuristic information in its selection of which path to expand. For each path on the frontier, A\* uses an estimate of the total path cost from the start node to a goal node constrained to follow that path initially. The estimated total path cost is the sum of the cost of the path found and the heuristic function $h(p)$, which estimates the cost from the end of $p$ to the goal.

In [ ]:
class AStarSearcher(Searcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """

    def __init__(self, problem):
        super().__init__(problem)

    def initialize_frontier(self):
        self.frontier = Frontier()

    def empty_frontier(self):
        return self.frontier.empty()

    def add_to_frontier(self, path):
        """add path to the frontier with the appropriate cost"""
        value = path.cost + self.problem.heuristic(path.end())
        self.frontier.add(path, value)

In [ ]:
from aipython.searchProblem import simple_problem1, simple_problem2, edgeless_problem, cyclic_delivery_problem, acyclic_delivery_problem
from IPython.display import display

s_astar = AStarSearcher(acyclic_delivery_problem)
s_astar.sleep_time = 0.2
s_astar.show_edge_costs = True
s_astar.show_node_heuristics = False
s_astar.text_size = 15
s_astar.search()
display(s_astar)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
s_astar.search()

## [3.7.2 A\* Search with Multiple Path Pruning](http://artint.info/2e/html/ArtInt2e.Ch3.S7.SS2.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=43) (page 43)

There is often more than one path to a node. If only one path is required, a search algorithm can prune from the frontier any path that leads to a node to which it has already found a path. Multiple-path pruning is implemented by maintaining an explored set (traditionally called closed list) of nodes that are at the end of paths that have been expanded. The explored set is initially empty. When a path $⟨n_0,…,n_k⟩$ is selected , if $n_k$ is already in the explored set, the path can be discarded. Otherwise, $n_k$ is added to the explored set, and the algorithm proceeds as before.

In [ ]:
from aipython.searchProblem import Path
from aispace2.jupyter.search import Displayable, visualize


class SearcherMPP(AStarSearcher):
    """returns a searcher for a problem.
    Paths can be found by repeatedly calling search().
    """
    def __init__(self, problem):
        super().__init__(problem)
        self.explored = set()

    @visualize
    def search(self):
        """returns next path from an element of problem's start nodes
        to a goal node. 
        Returns None if no path exists.
        """
        while not self.empty_frontier():
            path = self.frontier.pop()
            if path.end() not in self.explored:
                self.display(2, "Expanding:",path,"(cost:",path.cost,")")
                self.explored.add(path.end())
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):
                    self.display(1, self.num_expanded, "paths have been expanded and",
                            len(self.frontier), "paths remain in the frontier")
                    self.solution = path   # store the solution found
                    return path
                else:
                    neighs = self.problem.neighbors(path.end())
                    self.display(3,"Neighbors are", neighs)
                    for arc in neighs:
                        self.add_to_frontier(Path(path,arc))
                    self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",
                     self.num_expanded,"paths expanded.")

In [ ]:
from aipython.searchProblem import simple_problem1, simple_problem2, edgeless_problem, cyclic_delivery_problem, acyclic_delivery_problem
from IPython.display import display

s_mpp = SearcherMPP(simple_problem2)
s_mpp.sleep_time = 0.2
s_mpp.text_size = 15
s_mpp.search()
display(s_mpp)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
s_mpp.search()

## [3.8.1 Branch-and-bound Search](http://artint.info/2e/html/ArtInt2e.Ch3.S8.SS1.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=44) (page 44)

Depth-first branch-and-bound search is a way to combine the space saving of depth-first search with heuristic information for finding optimal paths. It is particularly applicable when there are many paths to a goal. As in A\* search, the heuristic function h$⁢(n)$ is non-negative and less than or equal to the cost of a lowest-cost path from n to a goal node.

The idea of a branch-and-bound search is to maintain the lowest-cost path to a goal found so far, and its cost. Suppose this cost is $\text{b⁢o⁢u⁢n⁢d}$. If the search encounters a path $p$ such that $\text{c⁢o⁢s⁢t}⁢(p)+h⁢(p)≥\text{b⁢o⁢u⁢n⁢d}$, path $p$ can be pruned. If a non-pruned path to a goal is found, it must be better than the previous best path. This new solution is remembered and b⁢o⁢u⁢n⁢d is set to the cost of this new solution. The searcher then proceeds to search for a better solution.


In [ ]:
from aipython.searchProblem import Path
from aispace2.jupyter.search import Displayable, visualize


class DF_branch_and_bound(Searcher):
    """returns a branch and bound searcher for a problem.    
    An optimal path with cost less than bound can be found by calling search()
    """
    def __init__(self, problem, bound=float("inf")):
        """creates a searcher than can be used with search() to find an optimal path.
        bound gives the initial bound. By default this is infinite - meaning there
        is no initial pruning due to depth bound
        """
        super().__init__(problem)
        self.best_path = None
        self.bound = bound

    @visualize
    def search(self):
        """returns an optimal solution to a problem with cost less than bound.
        returns None if there is no solution with cost less than bound."""
        self.frontier = [Path(self.problem.start_node())]
        self.num_expanded = 0
        while self.frontier:
            path = self.frontier.pop()
            if path.cost+self.problem.heuristic(path.end()) < self.bound:
                self.display(3,"Expanding:",path,"cost:",path.cost)
                self.num_expanded += 1
                if self.problem.is_goal(path.end()):
                    self.best_path = path
                    self.bound = path.cost
                    self.display(2,"New best path:",path," cost:",path.cost)
                    return
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path, arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"Number of paths expanded:",self.num_expanded)
        self.solution = self.best_path
        return self.best_path

In [ ]:
from aipython.searchProblem import simple_problem1, simple_problem2, edgeless_problem, cyclic_delivery_problem, acyclic_delivery_problem
from IPython.display import display

s_dfbb = DF_branch_and_bound(simple_problem2)
s_dfbb.sleep_time = 0.2
s_dfbb.text_size = 15
s_dfbb.search()
display(s_dfbb)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
s_dfbb.search()